# Import

In [1]:
# Data Handling
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import category_encoders as ce
from sklearn import preprocessing
from sklearn.decomposition import PCA


# Visualization
import matplotlib.pylab as plt
%matplotlib inline


# OS
import os
import time
import warnings; warnings.filterwarnings("ignore")
from tqdm import tqdm


# Modeling
from sklearn.model_selection import KFold
n_splits=5; seed = 42
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeClassifier, Lasso, ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.neural_network import MLPClassifier


# Hyperparameter Optimization
from sklearn.model_selection import RandomizedSearchCV


# Ensemble
from itertools import combinations
from sklearn.ensemble import VotingClassifier


# Evaluation
from sklearn.model_selection import cross_val_score
from scipy.stats.mstats import gmean
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


# Saving
import joblib

# Data Load

In [2]:
train = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/features12_tr.csv').drop('cust',axis=1)
test = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/features3_te.csv').drop('cust',axis=1)

In [3]:
print(train.shape)
display(train)

(18638, 81)


,mpd_nunique,favorite_pd,favorite_zon,favorite_loc,mean_buy_am,max_buy_am,num_visit,day_sum_buy_am,day_count_buy_am,buy_mean_end,...,frequency,monetary,R,F,M,RFMscore,weekend_holiday_use,buy_ct_sum,most_buy_cop_u,lpay_use_status
0,67.0,우유,Z17014,Z17,8073.0,175000.0,38,47376.578947,5.868421,10137.342857,...,223,1800310.0,4,5,4,435,223,319.0,A02,Lpay_X
1,31.0,임대매출,Z17004,Z17,33155.0,199000.0,14,151567.857143,4.571429,45270.000000,...,64,2121950.0,2,3,4,335,64,66.0,D02,Lpay_X
2,33.0,요구르트,Z10024,Z10,22617.0,460000.0,14,145393.571429,6.428571,23850.789474,...,90,2035510.0,4,4,4,400,90,107.0,정보없음,Lpay_X
3,106.0,스낵류,Z16009,Z16,20355.0,3100000.0,55,158032.072727,7.763636,10824.674074,...,427,8691764.0,4,5,5,485,427,538.0,D01,Lpay_Offline
4,20.0,맥주,Z17004,Z17,13024.0,143040.0,11,47360.909091,3.636364,3945.000000,...,40,520970.0,1,2,2,185,40,53.0,정보없음,Lpay_X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18633,0.0,정보없음,정보없음,정보없음,10200.0,23600.0,2,20400.000000,2.000000,0.000000,...,4,40800.0,1,1,1,100,4,0.0,C02,Lpay_X
18634,0.0,정보없음,정보없음,정보없음,3000.0,3000.0,1,3000.000000,1.000000,3000.000000,...,1,3000.0,3,1,1,130,1,0.0,C02,Lpay_X
18635,0.0,정보없음,정보없음,정보없음,8600.0,8600.0,1,8600.000000,1.000000,8600.000000,...,1,8600.0,1,1,1,100,1,0.0,D01,Lpay_X
18636,0.0,정보없음,정보없음,정보없음,10300.0,10300.0,1,10300.000000,1.000000,0.000000,...,1,10300.0,1,1,1,100,1,0.0,D02,Lpay_X


In [4]:
print(test.shape)
display(test)

(3616, 81)


,mpd_nunique,favorite_pd,favorite_zon,favorite_loc,mean_buy_am,max_buy_am,num_visit,day_sum_buy_am,day_count_buy_am,buy_mean_end,...,monetary,R,F,M,RFMscore,weekend_holiday_use,most_expensive,buy_ct_sum,most_buy_cop_u,lpay_use_status
0,3.0,냉동간편식,Z17018,Z17,7570.0,15000.0,2,15140.000000,2.000000,0.000000,...,30280.0,2,1,1,115,4,0,3.0,0,Lpay_X
1,30.0,일반담배,Z04003,Z04,5297.0,19459.0,22,12761.181818,2.409091,5370.714286,...,280746.0,5,4,3,365,53,0,64.0,0,Lpay_X
2,3.0,모바일상품권,점포없음,Z10,41905.0,85000.0,12,69841.666667,1.666667,15800.000000,...,838100.0,1,2,4,285,20,0,16.0,0,Lpay_Online
3,53.0,레토르트,Z10044,Z10,6342.0,39600.0,19,58083.157895,9.157895,6955.909091,...,1103580.0,3,5,4,420,174,0,204.0,0,Lpay_X
4,36.0,식당,Z06009,Z06,6554.0,33000.0,8,53252.625000,8.125000,7752.000000,...,426021.0,3,4,3,335,65,0,71.0,0,Lpay_X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,45.0,냉장간편식,Z10041,Z10,6277.0,167200.0,12,52831.916667,8.416667,3545.750000,...,633983.0,5,5,4,450,101,0,109.0,D01,Lpay_X
3612,40.0,냉장간편식,Z10027,Z10,6824.0,50000.0,5,126929.000000,18.600000,6130.703125,...,634645.0,3,5,4,420,93,0,112.0,0,Lpay_X
3613,62.0,요구르트,Z10041,Z10,5419.0,36490.0,22,41384.363636,7.636364,4912.888889,...,910456.0,4,5,4,435,168,0,200.0,0,Lpay_X
3614,44.0,가공유,Z11016,Z11,6182.0,89800.0,20,37094.800000,6.000000,3925.294118,...,741896.0,4,5,4,435,120,0,169.0,0,Lpay_X


# Preprocessing

In [5]:
pre_df = pd.concat([train,test]).reset_index(drop=True)

In [6]:
obj_col = ['favorite_pd','favorite_zon','favorite_loc','most_expensive','most_buying_weekday',
           'pdde_most_use_chnl','pdde_cop_c_most_use','cop_u_cop_c_most_use','most_buy_cop_u']

In [7]:
train.drop(obj_col,axis=1,inplace=True)
test.drop(obj_col,axis=1,inplace=True)

In [8]:
oh_f = pd.get_dummies(pre_df[obj_col])

In [9]:
max_col = oh_f.shape[1] # 차원축소하려는 데이터의 column의 개수
pca = PCA(n_components=max_col, random_state=0).fit(oh_f)

cumsum = np.cumsum(pca.explained_variance_ratio_) # 분산의 설명량을 누적합
num_col = np.argmax(cumsum >= 0.95) + 1 # 분산의 설명량이 99%이상 되는 차원의 수

In [10]:
# 차원축소
pca = PCA(n_components = num_col, random_state=0).fit_transform(oh_f)
pca_f = pd.DataFrame(pca)

In [11]:
pca_f

,0,1,2,3,4,5,6,7,8,9,...,182,183,184,185,186,187,188,189,190,191
0,0.191501,-0.112236,0.958531,-0.083342,-0.052312,-0.208869,0.471943,0.717727,0.423168,0.231068,...,-0.002334,0.017554,0.056797,-0.014682,-0.062779,-0.217095,-0.110391,0.034150,-0.059313,0.031181
1,-1.013464,-0.434413,0.593928,0.182361,-0.413093,0.310328,-0.939443,0.239130,0.004422,-0.015383,...,0.004791,-0.000611,-0.002588,-0.003958,0.000375,-0.008182,0.007107,0.000452,-0.007055,-0.006090
2,-0.187591,0.921284,-0.681609,0.131501,-0.992363,-0.669599,0.736022,-0.021560,0.515335,-0.181147,...,0.006375,-0.001140,0.007051,-0.002111,-0.017695,-0.016842,0.017394,-0.002291,0.019267,0.009348
3,1.196243,0.298568,0.051552,-0.312531,-0.584174,0.298376,0.401735,0.330778,-0.030702,-0.314669,...,-0.328090,0.404541,-0.419364,0.220959,-0.296112,-0.000859,-0.038513,-0.090912,-0.102636,-0.071757
4,-0.434874,1.016753,0.531726,0.008844,-0.678239,-0.521801,0.788062,-0.120568,0.317233,-0.635663,...,0.000443,-0.002720,-0.005127,0.002802,-0.004704,0.011221,0.001425,-0.001499,-0.003597,0.007229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22249,0.810683,-0.661437,-0.401230,-0.219604,0.271717,-0.763714,-0.263418,0.111403,0.222218,0.722943,...,0.270014,-0.158337,0.144582,0.192986,0.055531,0.107023,-0.252700,-0.268022,0.146454,-0.344971
22250,-0.179954,0.621145,-0.904105,-1.315196,-0.031151,-0.324641,0.237205,0.679879,0.097246,0.044555,...,0.225998,-0.154826,0.060391,0.207129,0.016105,0.180184,-0.308370,-0.268452,0.159126,-0.291957
22251,0.471635,0.325574,-0.810347,-1.141069,0.362733,-0.558081,-0.395960,-0.017563,-0.129607,0.245300,...,0.005724,0.011889,0.056675,-0.006410,-0.000555,-0.038089,0.070229,0.000866,-0.029900,-0.045544
22252,0.451303,0.430659,-0.383854,-1.375579,0.098317,-0.073960,0.262197,0.572523,-0.050728,-0.561449,...,-0.055626,0.044647,-0.064361,-0.008896,-0.002567,-0.016297,0.068690,-0.043468,0.027137,-0.024060


In [12]:
cf_tr = pca_f.iloc[:len(train),:].reset_index(drop=True)
cf_te = pca_f.iloc[len(train):,:].reset_index(drop=True)

In [13]:
train = pd.concat([train,cf_tr],axis=1)
test= pd.concat([test,cf_te],axis=1)

In [14]:
train

,mpd_nunique,mean_buy_am,max_buy_am,num_visit,day_sum_buy_am,day_count_buy_am,buy_mean_end,buy_mean_middle,buy_mean_begin,buy_count_end,...,182,183,184,185,186,187,188,189,190,191
0,67.0,8073.0,175000.0,38,47376.578947,5.868421,10137.342857,6870.746269,7329.720930,70,...,-0.002334,0.017554,0.056797,-0.014682,-0.062779,-0.217095,-0.110391,0.034150,-0.059313,0.031181
1,31.0,33155.0,199000.0,14,151567.857143,4.571429,45270.000000,9263.548387,58904.800000,8,...,0.004791,-0.000611,-0.002588,-0.003958,0.000375,-0.008182,0.007107,0.000452,-0.007055,-0.006090
2,33.0,22617.0,460000.0,14,145393.571429,6.428571,23850.789474,52458.571429,10388.421053,38,...,0.006375,-0.001140,0.007051,-0.002111,-0.017695,-0.016842,0.017394,-0.002291,0.019267,0.009348
3,106.0,20355.0,3100000.0,55,158032.072727,7.763636,10824.674074,36700.345679,9884.438462,135,...,-0.328090,0.404541,-0.419364,0.220959,-0.296112,-0.000859,-0.038513,-0.090912,-0.102636,-0.071757
4,20.0,13024.0,143040.0,11,47360.909091,3.636364,3945.000000,21647.894737,6007.692308,8,...,0.000443,-0.002720,-0.005127,0.002802,-0.004704,0.011221,0.001425,-0.001499,-0.003597,0.007229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18633,0.0,10200.0,23600.0,2,20400.000000,2.000000,0.000000,0.000000,10200.000000,0,...,-0.006790,-0.008050,-0.001821,-0.007212,-0.004545,-0.003622,-0.003338,-0.001202,-0.005560,-0.004881
18634,0.0,3000.0,3000.0,1,3000.000000,1.000000,3000.000000,0.000000,0.000000,1,...,-0.003576,-0.002594,-0.002604,-0.010327,-0.005345,-0.000919,-0.005212,-0.001396,-0.002299,-0.003878
18635,0.0,8600.0,8600.0,1,8600.000000,1.000000,8600.000000,0.000000,0.000000,1,...,-0.001377,-0.008257,-0.002246,-0.000600,-0.006694,-0.002844,-0.004169,0.002936,-0.003470,-0.004382
18636,0.0,10300.0,10300.0,1,10300.000000,1.000000,0.000000,0.000000,10300.000000,0,...,0.004238,-0.004537,0.000070,-0.002394,-0.004521,-0.001971,-0.003825,0.001195,-0.004558,-0.001882


In [15]:
test

,mpd_nunique,mean_buy_am,max_buy_am,num_visit,day_sum_buy_am,day_count_buy_am,buy_mean_end,buy_mean_middle,buy_mean_begin,buy_count_end,...,182,183,184,185,186,187,188,189,190,191
0,3.0,7570.0,15000.0,2,15140.000000,2.000000,0.000000,7570.000000,0.000000,0,...,0.016208,0.000939,-0.004713,0.003105,-0.001782,-0.014794,0.000767,0.000611,-0.003700,-0.009767
1,30.0,5297.0,19459.0,22,12761.181818,2.409091,5370.714286,6338.666667,4603.166667,14,...,-0.033455,0.005198,0.001442,-0.009418,-0.033322,0.058844,0.004954,0.006703,0.014404,0.013101
2,3.0,41905.0,85000.0,12,69841.666667,1.666667,15800.000000,85000.000000,29162.500000,6,...,-0.018144,-0.000203,-0.003465,0.019719,0.009172,0.028119,0.034630,0.000243,-0.040177,-0.006698
3,53.0,6342.0,39600.0,19,58083.157895,9.157895,6955.909091,5578.545455,6371.132075,66,...,-0.030698,0.026000,0.021097,0.004792,-0.029501,0.062577,0.027188,-0.001755,-0.002247,0.047913
4,36.0,6554.0,33000.0,8,53252.625000,8.125000,7752.000000,7062.157895,5953.333333,10,...,0.029004,0.017609,0.021805,0.016459,0.018973,-0.020913,0.016922,0.016655,-0.021638,-0.020944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,45.0,6277.0,167200.0,12,52831.916667,8.416667,3545.750000,14620.000000,4267.466667,20,...,0.270014,-0.158337,0.144582,0.192986,0.055531,0.107023,-0.252700,-0.268022,0.146454,-0.344971
3612,40.0,6824.0,50000.0,5,126929.000000,18.600000,6130.703125,0.000000,8354.482759,64,...,0.225998,-0.154826,0.060391,0.207129,0.016105,0.180184,-0.308370,-0.268452,0.159126,-0.291957
3613,62.0,5419.0,36490.0,22,41384.363636,7.636364,4912.888889,4400.379747,7766.954545,45,...,0.005724,0.011889,0.056675,-0.006410,-0.000555,-0.038089,0.070229,0.000866,-0.029900,-0.045544
3614,44.0,6182.0,89800.0,20,37094.800000,6.000000,3925.294118,9133.911111,4814.878049,34,...,-0.055626,0.044647,-0.064361,-0.008896,-0.002567,-0.016297,0.068690,-0.043468,0.027137,-0.024060


In [16]:
label_encoder=preprocessing.LabelEncoder()
X_train = train.drop('lpay_use_status',axis=1)
train_target = label_encoder.fit_transform(train['lpay_use_status'])
X_test = test.drop('lpay_use_status',axis=1)
test_target = label_encoder.transform(test['lpay_use_status'])

# Model

#### 1. 단일 모델 기본 성능 확인

In [17]:
# 단일모델 정의
knn_clf = KNeighborsClassifier(n_jobs=-1)
extra_clf = ExtraTreesClassifier(random_state=seed, n_jobs=-1)
gbm_clf = GradientBoostingClassifier(random_state=seed)
xgb_clf = XGBClassifier(random_state=seed, n_jobs=-1)
lgb_clf = LGBMClassifier(random_state=seed, n_jobs=-1)
cat_clf = CatBoostClassifier(random_state=seed, verbose=False)

clfs = [ridge_clf, knn_clf, extra_clf, gbm_clf, xgb_clf, lgb_clf, cat_clf]


# Cross_val_score 함수정의
def get_model_cv_prediction(model, feature_data, y_target):
    roc_auc_scores = cross_val_score(model, feature_data, y_target, scoring='roc_auc_ovr', cv=kfold, n_jobs=-1)
    avg_roc_auc = np.mean(roc_auc_scores)
    model = model.fit(feature_data,y_target)
    pred = model.predict_proba(X_test.values)#[:,1]
    test_roc = roc_auc_score(test_target,pred,multi_class='ovr',average='weighted')
    print(f'{model.__class__.__name__} 모델의 평균 AUC: {avg_roc_auc:.3f} test AUC : {test_roc:.3f}')


# 단일 모델별 평균 성능 출력
for clf in clfs:
    get_model_cv_prediction(clf, X_train.values, train_target)

KNeighborsClassifier 모델의 평균 AUC: 0.644 test AUC : 0.623
ExtraTreesClassifier 모델의 평균 AUC: 0.820 test AUC : 0.775
GradientBoostingClassifier 모델의 평균 AUC: 0.838 test AUC : 0.791
XGBClassifier 모델의 평균 AUC: 0.824 test AUC : 0.775
LGBMClassifier 모델의 평균 AUC: 0.833 test AUC : 0.786
CatBoostClassifier 모델의 평균 AUC: 0.836 test AUC : 0.791


#### 2. 랜덤서치를 활용한 모델 튜닝 

In [19]:
clfs = [
    (
        KNeighborsClassifier(),              
        {'n_neighbors': [3,5,7,9,11],        
         'weights': ['uniform','distance']}
    ),
    (
        ExtraTreesClassifier(),
        {'n_estimators': [100, 150, 200, 250, 300],
         'max_depth': [10, 12, 15, 17, 20],
         'max_features': [0.8, 0.85, 0.9, 0.95],
         'min_samples_split': [1, 2, 3, 4, 5],
         'min_samples_leaf': [1, 2, 3, 4, 5]}
    ),
    (
        GradientBoostingClassifier(),
        {
         'n_estimators': [100,300,500,1000],
         'learning_rate': [0.01,0.03,0.05,0.1],
         'max_depth': [3,5,6],
         'min_samples_leaf' : [3,5,7,9,10],
         'min_samples_split' : [2,4,6,8,10],
         'subsample' : [0.8,0.9,0.95,1]
         }
    ),
    (
        XGBClassifier(),
        {
        'n_estimators' :[100,200,300,400,500], # kaggle competition에서 best = 1000 
        'learning_rate': [0.01,0.03,0.05,0.1],
         'max_depth': [3,5,6],
         'colsample_bytree' :[0,0.1,0.3,0.5,0.7,0.9,1],
         'min_child_weight' :[1,3,5,6],
         'subsample' :[0.8,0.9,0.95,1],
         'objective' : ['clf:squarederror']
        }
    ),
    (
        LGBMClassifier(),
        {'n_estimators': [300,500,700,1000,1100],
         'learning_rate': [0.01,0.03,0.05,0.1],
         'max_depth': [3,5,7,9,10],
         #'boosting' : ['gbrt','dart'],
         'colsample_bytree' : [0,0.1,0.3,0.5,0.7,0.9,1],
         'subsample' :[0.8,0.9,0.95,1],
         'num_leaves' :[30,31,33,35,39,40]
         #'feature_fraction' : [0.1,0.3,0.5,0.7,0.9]
        }
    ),
    (
        CatBoostClassifier(),
        {'learning_rate': [0.05, 0.1, 0.2, 1, 1.5],
         'depth': [3, 5, 7, 9, 10],
         'iterations' : [500, 700, 1000, 1200],
         'verbose':[False]}
    )
]


RS_tuned_clfs = []  # 튜닝된 모델을 저장
for clf, param_grid in clfs:
    start = time.time()
    rand_search = RandomizedSearchCV(clf, param_grid, n_iter=20, scoring='roc_auc_ovr', 
                                     cv=kfold, random_state=seed, n_jobs=-1)
    rand_search.fit(X_train.values, train_target)
    clf_name = clf.__class__.__name__
    roc_score = rand_search.best_score_
    pred = rand_search.best_estimator_.predict_proba(X_test)
    test_roc = roc_auc_score(test_target,pred,multi_class='ovr',average='weighted')
    print(f'{clf_name:30s} mean_roc: {roc_score:.3f}, takes {time.time() - start:.1f} secs, test_roc: {test_roc:.3f}')
    RS_tuned_clfs.append((clf_name, rand_search.best_estimator_, roc_score,test_roc ))

KNeighborsClassifier           mean_roc: 0.677, takes 28.2 secs, test_roc: 0.653
ExtraTreesClassifier           mean_roc: 0.837, takes 993.1 secs, test_roc: 0.793
GradientBoostingClassifier     mean_roc: 0.840, takes 15970.7 secs, test_roc: 0.793
XGBClassifier                  mean_roc: 0.842, takes 2590.1 secs, test_roc: 0.797
LGBMClassifier                 mean_roc: 0.841, takes 548.2 secs, test_roc: 0.797
CatBoostClassifier             mean_roc: 0.843, takes 23002.6 secs, test_roc: 0.798


In [20]:
# 튜닝된 모델들의 결과
RS_tuned_clfs

[('KNeighborsClassifier',
  KNeighborsClassifier(n_neighbors=11, weights='distance'),
  0.6770889689746159,
  0.6525035520052103),
 ('ExtraTreesClassifier',
  ExtraTreesClassifier(max_depth=15, max_features=0.95, min_samples_leaf=5,
                       n_estimators=200),
  0.8371160967954158,
  0.7925099283520295),
 ('GradientBoostingClassifier',
  GradientBoostingClassifier(learning_rate=0.05, min_samples_leaf=3,
                             min_samples_split=6, subsample=0.9),
  0.8402746984673897,
  0.7926929110211274),
 ('XGBClassifier',
  XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='',
                learning_rate=0.03, max_bin=256, max_cat_to_onehot=4,
                max

 - 튜닝된 모델들 저장

In [21]:
now = datetime.datetime.now()
nowDate = now.strftime('%Y-%m-%d')
joblib.dump(RS_tuned_clfs,f'../3.로티로리_데이터 및 모델 세이브 파일/RS_tuned_clfs_{nowDate}_pca_roc.pkl')

['../models/RS_tuned_clfs_2022-08-12_pca_roc.pkl']

In [22]:
# 튜닝된 모델들 로드
RS_tuned_clfs = joblib.load('../3.로티로리_데이터 및 모델 세이브 파일/RS_tuned_clfs_2022-08-12_pca_roc.pkl')

#### 3. Catboost pred

In [29]:
pred = pd.DataFrame(RS_tuned_clfs[5][1].predict_proba(X_test))

In [31]:
pred.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Catboost_pred_df.csv',index=False)

#### 4. 튜닝된 모델들로 Voting 앙상블

In [32]:
# 모델 훈련 함수 정의
def return_fitted_model(model, train, target):
    acc_scores = []
    for iter_count, (train_idx, valid_idx) in enumerate(kfold.split(train, target)):

        X_train, X_valid = train[train_idx], train[valid_idx]
        y_train, y_valid = target[train_idx], target[valid_idx]

        model.fit(X_train, y_train)

        pred = model.predict(X_valid)
        acc_score = accuracy_score(y_valid, pred)
        acc_scores.append(acc_score)
    return model, np.mean(acc_scores)

# ex: extra_reg, extra_reg_score = return_fitted_model(extra_reg, train, target)
# print(f'모델의 평균 성능:  {extra_reg_score:.3f}')

 - 튜닝된 모델들 중 성능이 비슷하고 좋은 4개의 모델에 대해서 앙상블을 진행함

In [33]:
selected_clf = [
    #'RidgeClassifier',
    #'MLPClassifier',
    #'KNeighborsRegressor', 
    'ExtraTreesClassifier',
    #'CatBoostClassifier', 
    #'GradientBoostingClassifier', 
    'XGBClassifier',
    'LGBMClassifier'
]
models_for_ensemble = [(clf[0], clf[1]) for clf in RS_tuned_clfs if clf[0] in selected_clf]

 - 모든 조합에 대해서 앙상블을 수행

In [42]:
# 3개의 모델 중 2개씩 averaging, 3개씩 averging 모두 해보기
start = time.time()
best_avg_score = np.inf
bams = []
for model_nums in range(2, len(models_for_ensemble ) + 1):
    for avg_estimator in (combinations(models_for_ensemble , model_nums)):
        avg_reg = VotingClassifier(estimators = avg_estimator, n_jobs=-1,voting='soft')
        avg_model, avg_score = return_fitted_model(avg_reg, X_train.values, train_target)
        print(f'{"●".join([clf_name for clf_name, _, in avg_estimator])}: {avg_score:.3f}')
        if avg_score >= best_avg_score:
            best_avg_score = avg_score
            best_avg_reg = avg_model
        bams.append(avg_model)
print(time.time() - start)

ExtraTreesClassifier●XGBClassifier: 0.724
ExtraTreesClassifier●LGBMClassifier: 0.724
XGBClassifier●LGBMClassifier: 0.724
ExtraTreesClassifier●XGBClassifier●LGBMClassifier: 0.724
2769.087702512741


 - 앙상블 모델 저장

In [47]:
now = datetime.datetime.now()
nowDate = now.strftime('%Y-%m-%d')
joblib.dump(bams[1],f'../3.로티로리_데이터 및 모델 세이브 파일/Average_Ensemble_{nowDate}_voting.pkl')

['../models/Average_Ensemble_2022-08-12_voting.pkl']

In [48]:
# 앙상블 모델 로드
best_avg_reg = joblib.load('../3.로티로리_데이터 및 모델 세이브 파일/Average_Ensemble_2022-08-12_voting.pkl')

- model 성능

In [45]:
pred = pd.DataFrame(bams[1].predict_proba(X_test))
print('Voting Ensemble roc_auc',roc_auc_score(test_target,pred,multi_class='ovr',average='weighted'))

Voting Ensemble roc_auc 0.7963545571595773


#### 5. 최종 서브미션 저장

In [49]:
pred.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/ML_Voting_pred_df.csv',index=False)